In [104]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader,random_split
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau
import torchmetrics.functional as metrics
import os
import shutil
from torchvision import transforms
from PIL import Image

In [105]:
# 학습 검증용 데이터
folder_path = '../train/train'
target_data = []
img_data = []
for encoding_label,label in enumerate(os.listdir(folder_path)):
    label_path = os.path.join(folder_path, label)
    if os.path.isdir(label_path):  # 디렉토리인 경우에만 진입
        for img in os.listdir(folder_path+'/'+label):
            image_path = os.path.join(folder_path,label,img)
            if os.path.isfile(image_path):  # 파일인 경우에만 진입
                with open(image_path, 'rb') as file:
                    image = Image.open(file)
                    width, height = image.size
                    if width == 48 and height == 48:
                        image_array = np.array(image)
                        target_data.append(encoding_label)
                        img_data.append(image_array)

In [106]:
# 테스트용 데이터
folder_path = '../test/test'
target_test = []
img_test = []
for encoding_label,label in enumerate(os.listdir(folder_path)):
    label_path = os.path.join(folder_path, label)
    if os.path.isdir(label_path):  # 디렉토리인 경우에만 진입
        for img in os.listdir(folder_path+'/'+label):
            image_path = os.path.join(folder_path,label,img)
            if os.path.isfile(image_path):  # 파일인 경우에만 진입
                with open(image_path, 'rb') as file:
                    image = Image.open(file)
                    width, height = image.size
                    if width == 48 and height == 48:
                        image_array = np.array(image)
                        target_test.append(encoding_label)
                        img_test.append(image_array)

In [107]:
pd.Series(target_test).value_counts().sort_index()

0    1774
1    1247
2    1024
3     831
4    1233
5     958
6     111
Name: count, dtype: int64

In [108]:
os.listdir(folder_path)

['happy', 'sad', 'fear', 'surprise', 'neutral', 'angry', 'disgust']

In [109]:
pd.Series(target_data).value_counts().sort_index()

0    7215
2    4830
3    4097
4    3171
5    4965
6    3995
7     436
Name: count, dtype: int64

In [110]:
image = Image.open("../test/test/angry/PrivateTest_1488292.jpg")

# 이미지의 너비와 높이 확인
width, height = image.size
print("이미지의 너비:", width)
print("이미지의 높이:", height)

# 이미지 크기 48 * 48

이미지의 너비: 48
이미지의 높이: 48


In [111]:
# 이미지 데이터 정규화
x_data = np.array(img_data)/255.
x_data = x_data.reshape((-1,48*48))
print(x_data.shape)

(28709, 2304)


In [112]:
# 원핫 인코딩
# from sklearn.preprocessing import OneHotEncoder
# y_data = OneHotEncoder(sparse_output=False).fit_transform(np.array(target_data).reshape(-1,1))

In [113]:
# pd.DataFrame(x_data).value_counts()

In [114]:
target_data = pd.Series(target_data).replace({0:3, 5:4, 2:5, 3:2, 6:0, 4:6, 7:1})
target_data.value_counts()

3    7215
4    4965
5    4830
2    4097
0    3995
6    3171
1     436
Name: count, dtype: int64

In [115]:
pd.Series(target_test).value_counts()

0    1774
1    1247
4    1233
2    1024
5     958
3     831
6     111
Name: count, dtype: int64

In [116]:
target_test = pd.Series(target_test).replace({0:3, 5:4, 2:5, 3:2, 6:0, 4:6, 7:1})
target_test.value_counts()

3    1774
1    1247
6    1233
5    1024
4     958
2     831
0     111
Name: count, dtype: int64

In [117]:
# 데이터 클래스 생성
class DLdataset(Dataset):
    
    def __init__(self,x_data,y_data):
        super().__init__()
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)
        
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self,idx):
        return self.feature[idx], self.target[idx]

In [118]:
# 데이터셋 생성
dataset = DLdataset(x_data,target_data)

In [ ]:
norm_test_target = np.array(test_target_data)
norm_test_feature = np.array(test_img_data)/255.

In [ ]:
testDS= DLDataset(norm_test_feature, norm_test_target)
testDL = DataLoader(testDS, batch_size=batch, drop_last=True)


In [ ]:
pd.Series(target_test).value_counts().sort_index()

0    1774
1    1247
2    1024
3     831
4    1233
5     958
6     111
Name: count, dtype: int64

In [ ]:
os.listdir(folder_path)

['happy', 'sad', 'fear', 'surprise', 'neutral', 'angry', 'disgust']

In [ ]:
pd.Series(target_data).value_counts().sort_index()

0    7215
2    4830
3    4097
4    3171
5    4965
6    3995
7     436
Name: count, dtype: int64

In [ ]:
image = Image.open("../test/test/angry/PrivateTest_1488292.jpg")

# 이미지의 너비와 높이 확인
width, height = image.size
print("이미지의 너비:", width)
print("이미지의 높이:", height)

# 이미지 크기 48 * 48

이미지의 너비: 48
이미지의 높이: 48


In [ ]:
# 이미지 데이터 정규화
x_data = np.array(img_data)/255.
x_data = x_data.reshape((-1,48*48))
print(x_data.shape)

(28709, 2304)


In [ ]:
# 원핫 인코딩
# from sklearn.preprocessing import OneHotEncoder
# y_data = OneHotEncoder(sparse_output=False).fit_transform(np.array(target_data).reshape(-1,1))

In [ ]:
# pd.DataFrame(x_data).value_counts()

In [ ]:
target_data = pd.Series(target_data).replace({0:3, 5:4, 2:5, 3:2, 6:0, 4:6, 7:1})
target_data.value_counts()

3    7215
4    4965
5    4830
2    4097
0    3995
6    3171
1     436
Name: count, dtype: int64

In [ ]:
pd.Series(target_test).value_counts()

0    1774
1    1247
4    1233
2    1024
5     958
3     831
6     111
Name: count, dtype: int64

In [ ]:
target_test = pd.Series(target_test).replace({0:3, 5:4, 2:5, 3:2, 6:0, 4:6, 7:1})
target_test.value_counts()

3    1774
1    1247
6    1233
5    1024
4     958
2     831
0     111
Name: count, dtype: int64

In [ ]:
# 데이터 클래스 생성
class DLdataset(Dataset):
    
    def __init__(self,x_data,y_data):
        super().__init__()
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)
        
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self,idx):
        return self.feature[idx], self.target[idx]

In [ ]:
# 데이터셋 생성
dataset = DLdataset(x_data,target_data)

In [ ]:
# 학습용, 검증용 데이터 준비
seed = torch.Generator().manual_seed(42)
trainDS, validDS = random_split(dataset, [0.8,0.2], generator=seed)

In [ ]:
# 배치사이즈 32
BATCH = 32
trainDL = DataLoader(trainDS, batch_size=BATCH)
validDL = DataLoader(validDS, batch_size=BATCH)



In [ ]:
# 모델 클래스 정의
class Model(nn.Module):
    
    def __init__(self, IN, OUT):
        super().__init__()
        self.input = nn.Linear(IN, 128) 
        self.af = nn.ReLU()
        self.hidden = nn.Linear(128, 32)
        self.output = nn.Linear(32, OUT)
        
    def forward(self, x):
        y = self.input(x)
        y = self.af(y)
        y = self.hidden(y)
        y = self.af(y)
        y = self.output(y)
        
        return y

In [ ]:
# 학습 준비

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

EPOCHS = 100

IN = dataset.feature.shape[1]
OUT = pd.Series(target_data).nunique()

# 모델 생성
model  = Model(IN,OUT)

# 손실함수
LF = nn.CrossEntropyLoss().to(DEVICE)

# 옵티마이저
OPTIMIZER = torch.optim.Adam(model.parameters())

# 스케줄러
SCHEDULER = ReduceLROnPlateau(OPTIMIZER, mode = 'min', patience = 3)

In [ ]:
def training(dataLoader):
    
    model.train()
    train_report=[[], []]
    for (feature, target) in dataLoader:

        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        # 학습
        pre_target = model(feature)
        
        # 손실계산
        loss = LF(pre_target, target)
        train_report[0].append(loss)
  
        # 성능 평가
        acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
        train_report[1].append(acc)
        
        # W,b업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

    loss_score = sum(train_report[0])/BATCH
    acc_score = sum(train_report[1])/BATCH
    print(f'[Train loss] ==> {loss_score}    [Train Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [ ]:
training(trainDL)

[Train loss] ==> 40.47188186645508    [Train Accuracy] ==> 5.694986820220947


(tensor(40.4719, grad_fn=<DivBackward0>), tensor(5.6950))

In [ ]:
def testing(dataLoader):
    
    model.eval()
    
    with torch.no_grad():
        test_report=[[], []]
        for (feature, target)  in dataLoader:
            # 배치크기만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            
            # 학습
            pre_target = model(feature)
            
            # 손실계산
            loss = LF(pre_target, target)
            test_report[0].append(loss)
      
            # 성능 평가
            acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
            test_report[1].append(acc)
    
        loss_score = sum(test_report[0])/BATCH
        acc_score = sum(test_report[1])/BATCH

    print(f'[Test loss] ==> {loss_score}    [Test Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [ ]:
testing(validDL)

[Test loss] ==> 9.980865478515625    [Test Accuracy] ==> 1.5150991678237915


(tensor(9.9809), tensor(1.5151))

In [ ]:
def predicting(dataLoader,n):

    model.eval()

    correct = 0
    total = 0

    for idx in range(len(dataset)):
        img, ytrue = dataset[idx][0], dataset[idx][1]

        with torch.no_grad():
            ypre = model(img.unsqueeze(0))
            ypre = torch.argmax(ypre, dim=1).item()

            if ypre == ytrue:
                correct += 1

            total += 1
            if idx < n :
                plt.imshow(dataset[idx][0].numpy().reshape(28, 28), cmap='gray_r')
                plt.title(f'[{idx+1}] True {ytrue} / Predict {ypre}')
                plt.xticks([])
                plt.yticks([])
                plt.show()

    accuracy = correct / total
    print(f'Accuracy: {accuracy:.2f}')
    return accuracy

In [ ]:
min_loss = 100.0  # 초기 최소 손실 설정
cnt = 0
for eps in range(EPOCHS):
    print(f'[{eps+1}/{EPOCHS}]')
    # 학습
    train_loss, train_acc = training(trainDL)

    # 검증
    val_loss, val_acc = testing(validDL)
    
    # 최소 손실 업데이트
    if val_loss < min_loss:
        min_loss = val_loss
        cnt = 0
    else:
        cnt+=1

    # 조기 종료 기능 => 조건 : val_loss가 지정된 횟수 이상 개선이 안되면 학습 종료
    if SCHEDULER.num_bad_epochs >= SCHEDULER.patience or cnt >= 5:
        print(f"Early stopping at epoch {eps}")
        break

[1/100]
[Train loss] ==> 38.64373779296875    [Train Accuracy] ==> 6.989908695220947
[Test loss] ==> 9.523439407348633    [Test Accuracy] ==> 1.8919771909713745
[2/100]
[Train loss] ==> 37.848960876464844    [Train Accuracy] ==> 7.472005367279053
[Test loss] ==> 9.499372482299805    [Test Accuracy] ==> 1.859299898147583
[3/100]
[Train loss] ==> 37.381629943847656    [Train Accuracy] ==> 7.737630367279053
[Test loss] ==> 9.358746528625488    [Test Accuracy] ==> 1.944260835647583
[4/100]
[Train loss] ==> 36.993003845214844    [Train Accuracy] ==> 7.902669429779053
[Test loss] ==> 9.287851333618164    [Test Accuracy] ==> 1.947190523147583
[5/100]
[Train loss] ==> 36.66273498535156    [Train Accuracy] ==> 8.098958015441895
[Test loss] ==> 9.278766632080078    [Test Accuracy] ==> 1.9681490659713745
[6/100]
[Train loss] ==> 36.39751434326172    [Train Accuracy] ==> 8.179036140441895
[Test loss] ==> 9.230303764343262    [Test Accuracy] ==> 1.9827975034713745
[7/100]
[Train loss] ==> 36.176876

KeyboardInterrupt: 

In [119]:
# 학습용, 검증용 데이터 준비
seed = torch.Generator().manual_seed(42)
trainDS, validDS = random_split(dataset, [0.8,0.2], generator=seed)

In [120]:
# 배치사이즈 32
BATCH = 32
trainDL = DataLoader(trainDS, batch_size=BATCH)
validDL = DataLoader(validDS, batch_size=BATCH)



NameError: name 'DLDataset' is not defined

In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader,random_split
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau
import torchmetrics.functional as metrics
import os
import shutil
from torchvision import transforms
from PIL import Image

In [ ]:
# 학습 검증용 데이터
folder_path = '../train/train'
target_data = []
img_data = []
for encoding_label,label in enumerate(os.listdir(folder_path)):
    label_path = os.path.join(folder_path, label)
    if os.path.isdir(label_path):  # 디렉토리인 경우에만 진입
        for img in os.listdir(folder_path+'/'+label):
            image_path = os.path.join(folder_path,label,img)
            if os.path.isfile(image_path):  # 파일인 경우에만 진입
                with open(image_path, 'rb') as file:
                    image = Image.open(file)
                    width, height = image.size
                    if width == 48 and height == 48:
                        image_array = np.array(image)
                        target_data.append(encoding_label)
                        img_data.append(image_array)

In [ ]:
# 테스트용 데이터
folder_path = '../test/test'
target_test = []
img_test = []
for encoding_label,label in enumerate(os.listdir(folder_path)):
    label_path = os.path.join(folder_path, label)
    if os.path.isdir(label_path):  # 디렉토리인 경우에만 진입
        for img in os.listdir(folder_path+'/'+label):
            image_path = os.path.join(folder_path,label,img)
            if os.path.isfile(image_path):  # 파일인 경우에만 진입
                with open(image_path, 'rb') as file:
                    image = Image.open(file)
                    width, height = image.size
                    if width == 48 and height == 48:
                        image_array = np.array(image)
                        target_test.append(encoding_label)
                        img_test.append(image_array)

In [ ]:
pd.Series(target_test).value_counts().sort_index()

0    1774
1    1247
2    1024
3     831
4    1233
5     958
6     111
Name: count, dtype: int64

In [ ]:
os.listdir(folder_path)

['happy', 'sad', 'fear', 'surprise', 'neutral', 'angry', 'disgust']

In [ ]:
pd.Series(target_data).value_counts().sort_index()

0    7215
2    4830
3    4097
4    3171
5    4965
6    3995
7     436
Name: count, dtype: int64

In [ ]:
image = Image.open("../test/test/angry/PrivateTest_1488292.jpg")

# 이미지의 너비와 높이 확인
width, height = image.size
print("이미지의 너비:", width)
print("이미지의 높이:", height)

# 이미지 크기 48 * 48

이미지의 너비: 48
이미지의 높이: 48


In [ ]:
# 이미지 데이터 정규화
x_data = np.array(img_data)/255.
x_data = x_data.reshape((-1,48*48))
print(x_data.shape)

(28709, 2304)


In [ ]:
# 원핫 인코딩
# from sklearn.preprocessing import OneHotEncoder
# y_data = OneHotEncoder(sparse_output=False).fit_transform(np.array(target_data).reshape(-1,1))

In [ ]:
# pd.DataFrame(x_data).value_counts()

In [ ]:
target_data = pd.Series(target_data).replace({0:3, 5:4, 2:5, 3:2, 6:0, 4:6, 7:1})
target_data.value_counts()

3    7215
4    4965
5    4830
2    4097
0    3995
6    3171
1     436
Name: count, dtype: int64

In [ ]:
pd.Series(target_test).value_counts()

0    1774
1    1247
4    1233
2    1024
5     958
3     831
6     111
Name: count, dtype: int64

In [ ]:
target_test = pd.Series(target_test).replace({0:3, 5:4, 2:5, 3:2, 6:0, 4:6, 7:1})
target_test.value_counts()

3    1774
1    1247
6    1233
5    1024
4     958
2     831
0     111
Name: count, dtype: int64

In [ ]:
# 데이터 클래스 생성
class DLdataset(Dataset):
    
    def __init__(self,x_data,y_data):
        super().__init__()
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)
        
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self,idx):
        return self.feature[idx], self.target[idx]

In [ ]:
# 데이터셋 생성
dataset = DLdataset(x_data,target_data)

In [ ]:
# 학습용, 검증용 데이터 준비
seed = torch.Generator().manual_seed(42)
trainDS, validDS = random_split(dataset, [0.8,0.2], generator=seed)

In [ ]:
# 배치사이즈 32
BATCH = 32
trainDL = DataLoader(trainDS, batch_size=BATCH)
validDL = DataLoader(validDS, batch_size=BATCH)



In [ ]:
# 모델 클래스 정의
class Model(nn.Module):
    
    def __init__(self, IN, OUT):
        super().__init__()
        self.input = nn.Linear(IN, 128) 
        self.af = nn.ReLU()
        self.hidden = nn.Linear(128, 32)
        self.output = nn.Linear(32, OUT)
        
    def forward(self, x):
        y = self.input(x)
        y = self.af(y)
        y = self.hidden(y)
        y = self.af(y)
        y = self.output(y)
        
        return y

In [ ]:
# 학습 준비

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

EPOCHS = 100

IN = dataset.feature.shape[1]
OUT = pd.Series(target_data).nunique()

# 모델 생성
model  = Model(IN,OUT)

# 손실함수
LF = nn.CrossEntropyLoss().to(DEVICE)

# 옵티마이저
OPTIMIZER = torch.optim.Adam(model.parameters())

# 스케줄러
SCHEDULER = ReduceLROnPlateau(OPTIMIZER, mode = 'min', patience = 3)

In [ ]:
def training(dataLoader):
    
    model.train()
    train_report=[[], []]
    for (feature, target) in dataLoader:

        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        # 학습
        pre_target = model(feature)
        
        # 손실계산
        loss = LF(pre_target, target)
        train_report[0].append(loss)
  
        # 성능 평가
        acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
        train_report[1].append(acc)
        
        # W,b업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

    loss_score = sum(train_report[0])/BATCH
    acc_score = sum(train_report[1])/BATCH
    print(f'[Train loss] ==> {loss_score}    [Train Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [ ]:
training(trainDL)

[Train loss] ==> 40.47188186645508    [Train Accuracy] ==> 5.694986820220947


(tensor(40.4719, grad_fn=<DivBackward0>), tensor(5.6950))

In [ ]:
def testing(dataLoader):
    
    model.eval()
    
    with torch.no_grad():
        test_report=[[], []]
        for (feature, target)  in dataLoader:
            # 배치크기만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            
            # 학습
            pre_target = model(feature)
            
            # 손실계산
            loss = LF(pre_target, target)
            test_report[0].append(loss)
      
            # 성능 평가
            acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
            test_report[1].append(acc)
    
        loss_score = sum(test_report[0])/BATCH
        acc_score = sum(test_report[1])/BATCH

    print(f'[Test loss] ==> {loss_score}    [Test Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [ ]:
testing(validDL)

[Test loss] ==> 9.980865478515625    [Test Accuracy] ==> 1.5150991678237915


(tensor(9.9809), tensor(1.5151))

In [ ]:
def predicting(dataLoader,n):

    model.eval()

    correct = 0
    total = 0

    for idx in range(len(dataset)):
        img, ytrue = dataset[idx][0], dataset[idx][1]

        with torch.no_grad():
            ypre = model(img.unsqueeze(0))
            ypre = torch.argmax(ypre, dim=1).item()

            if ypre == ytrue:
                correct += 1

            total += 1
            if idx < n :
                plt.imshow(dataset[idx][0].numpy().reshape(28, 28), cmap='gray_r')
                plt.title(f'[{idx+1}] True {ytrue} / Predict {ypre}')
                plt.xticks([])
                plt.yticks([])
                plt.show()

    accuracy = correct / total
    print(f'Accuracy: {accuracy:.2f}')
    return accuracy

In [ ]:
min_loss = 100.0  # 초기 최소 손실 설정
cnt = 0
for eps in range(EPOCHS):
    print(f'[{eps+1}/{EPOCHS}]')
    # 학습
    train_loss, train_acc = training(trainDL)

    # 검증
    val_loss, val_acc = testing(validDL)
    
    # 최소 손실 업데이트
    if val_loss < min_loss:
        min_loss = val_loss
        cnt = 0
    else:
        cnt+=1

    # 조기 종료 기능 => 조건 : val_loss가 지정된 횟수 이상 개선이 안되면 학습 종료
    if SCHEDULER.num_bad_epochs >= SCHEDULER.patience or cnt >= 5:
        print(f"Early stopping at epoch {eps}")
        break

[1/100]
[Train loss] ==> 38.64373779296875    [Train Accuracy] ==> 6.989908695220947
[Test loss] ==> 9.523439407348633    [Test Accuracy] ==> 1.8919771909713745
[2/100]
[Train loss] ==> 37.848960876464844    [Train Accuracy] ==> 7.472005367279053
[Test loss] ==> 9.499372482299805    [Test Accuracy] ==> 1.859299898147583
[3/100]
[Train loss] ==> 37.381629943847656    [Train Accuracy] ==> 7.737630367279053
[Test loss] ==> 9.358746528625488    [Test Accuracy] ==> 1.944260835647583
[4/100]
[Train loss] ==> 36.993003845214844    [Train Accuracy] ==> 7.902669429779053
[Test loss] ==> 9.287851333618164    [Test Accuracy] ==> 1.947190523147583
[5/100]
[Train loss] ==> 36.66273498535156    [Train Accuracy] ==> 8.098958015441895
[Test loss] ==> 9.278766632080078    [Test Accuracy] ==> 1.9681490659713745
[6/100]
[Train loss] ==> 36.39751434326172    [Train Accuracy] ==> 8.179036140441895
[Test loss] ==> 9.230303764343262    [Test Accuracy] ==> 1.9827975034713745
[7/100]
[Train loss] ==> 36.176876

KeyboardInterrupt: 

In [ ]:
# 모델 클래스 정의
class Model(nn.Module):
    
    def __init__(self, IN, OUT):
        super().__init__()
        self.input = nn.Linear(IN, 128) 
        self.af = nn.ReLU()
        self.hidden = nn.Linear(128, 32)
        self.output = nn.Linear(32, OUT)
        
    def forward(self, x):
        y = self.input(x)
        y = self.af(y)
        y = self.hidden(y)
        y = self.af(y)
        y = self.output(y)
        
        return y

In [ ]:
# 학습 준비

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

EPOCHS = 100

IN = dataset.feature.shape[1]
OUT = pd.Series(target_data).nunique()

# 모델 생성
model  = Model(IN,OUT)

# 손실함수
LF = nn.CrossEntropyLoss().to(DEVICE)

# 옵티마이저
OPTIMIZER = torch.optim.Adam(model.parameters())

# 스케줄러
SCHEDULER = ReduceLROnPlateau(OPTIMIZER, mode = 'min', patience = 3)

In [ ]:
def training(dataLoader):
    
    model.train()
    train_report=[[], []]
    for (feature, target) in dataLoader:

        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        # 학습
        pre_target = model(feature)
        
        # 손실계산
        loss = LF(pre_target, target)
        train_report[0].append(loss)
  
        # 성능 평가
        acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
        train_report[1].append(acc)
        
        # W,b업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

    loss_score = sum(train_report[0])/BATCH
    acc_score = sum(train_report[1])/BATCH
    print(f'[Train loss] ==> {loss_score}    [Train Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [ ]:
training(trainDL)

[Train loss] ==> 40.47188186645508    [Train Accuracy] ==> 5.694986820220947


(tensor(40.4719, grad_fn=<DivBackward0>), tensor(5.6950))

In [ ]:
def testing(dataLoader):
    
    model.eval()
    
    with torch.no_grad():
        test_report=[[], []]
        for (feature, target)  in dataLoader:
            # 배치크기만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            
            # 학습
            pre_target = model(feature)
            
            # 손실계산
            loss = LF(pre_target, target)
            test_report[0].append(loss)
      
            # 성능 평가
            acc = metrics.accuracy(pre_target.argmax(dim=1), target, task = 'multiclass',num_classes=OUT)
            test_report[1].append(acc)
    
        loss_score = sum(test_report[0])/BATCH
        acc_score = sum(test_report[1])/BATCH

    print(f'[Test loss] ==> {loss_score}    [Test Accuracy] ==> {acc_score}')
    return loss_score, acc_score

In [ ]:
testing(validDL)

[Test loss] ==> 9.980865478515625    [Test Accuracy] ==> 1.5150991678237915


(tensor(9.9809), tensor(1.5151))

In [ ]:
def predicting(dataLoader,n):

    model.eval()

    correct = 0
    total = 0

    for idx in range(len(dataset)):
        img, ytrue = dataset[idx][0], dataset[idx][1]

        with torch.no_grad():
            ypre = model(img.unsqueeze(0))
            ypre = torch.argmax(ypre, dim=1).item()

            if ypre == ytrue:
                correct += 1

            total += 1
            if idx < n :
                plt.imshow(dataset[idx][0].numpy().reshape(28, 28), cmap='gray_r')
                plt.title(f'[{idx+1}] True {ytrue} / Predict {ypre}')
                plt.xticks([])
                plt.yticks([])
                plt.show()

    accuracy = correct / total
    print(f'Accuracy: {accuracy:.2f}')
    return accuracy

In [ ]:
predicting(testDL, 5)

NameError: name 'testDL' is not defined

In [ ]:
min_loss = 100.0  # 초기 최소 손실 설정
cnt = 0
for eps in range(EPOCHS):
    print(f'[{eps+1}/{EPOCHS}]')
    # 학습
    train_loss, train_acc = training(trainDL)

    # 검증
    val_loss, val_acc = testing(validDL)
    
    # 최소 손실 업데이트
    if val_loss < min_loss:
        min_loss = val_loss
        cnt = 0
    else:
        cnt+=1

    # 조기 종료 기능 => 조건 : val_loss가 지정된 횟수 이상 개선이 안되면 학습 종료
    if SCHEDULER.num_bad_epochs >= SCHEDULER.patience or cnt >= 5:
        print(f"Early stopping at epoch {eps}")
        break

[1/100]
[Train loss] ==> 38.64373779296875    [Train Accuracy] ==> 6.989908695220947
[Test loss] ==> 9.523439407348633    [Test Accuracy] ==> 1.8919771909713745
[2/100]
[Train loss] ==> 37.848960876464844    [Train Accuracy] ==> 7.472005367279053
[Test loss] ==> 9.499372482299805    [Test Accuracy] ==> 1.859299898147583
[3/100]
[Train loss] ==> 37.381629943847656    [Train Accuracy] ==> 7.737630367279053
[Test loss] ==> 9.358746528625488    [Test Accuracy] ==> 1.944260835647583
[4/100]
[Train loss] ==> 36.993003845214844    [Train Accuracy] ==> 7.902669429779053
[Test loss] ==> 9.287851333618164    [Test Accuracy] ==> 1.947190523147583
[5/100]
[Train loss] ==> 36.66273498535156    [Train Accuracy] ==> 8.098958015441895
[Test loss] ==> 9.278766632080078    [Test Accuracy] ==> 1.9681490659713745
[6/100]
[Train loss] ==> 36.39751434326172    [Train Accuracy] ==> 8.179036140441895
[Test loss] ==> 9.230303764343262    [Test Accuracy] ==> 1.9827975034713745
[7/100]
[Train loss] ==> 36.176876

KeyboardInterrupt: 